In [7]:
import dill
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode()

In [24]:
with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_100samples_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_100_R = dill.load(f)

with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_500samples_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_500_R = dill.load(f)

with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_500samples_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_1000_R = dill.load(f)
    
with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_5000samples_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_5000_R = dill.load(f)
    
with open('/home/lucas/tesis2/test_outputs/fatsfeatures_pca2_50samples_per_class_20results_per_query.dill', 'rb') as f:
    fats_pca2 = dill.load(f)
    
with open('/home/lucas/tesis2/test_outputs/macfeatures_50samples_per_class_20results_per_query.dill', 'rb') as f:
    mac_feats = dill.load(f)
    
with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_100samples_semistdTrue_stdFalse_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_100_R_semi = dill.load(f)

with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_500samples_semistdTrue_stdFalse_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_500_R_semi = dill.load(f)
    
with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_1000samples_semistdTrue_stdFalse_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_1000_R_semi = dill.load(f)
    
with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_5000samples_semistdTrue_stdFalse_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_5000_R_semi = dill.load(f)
    
    

with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_500samples_semistdFalse_stdTrue_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_500_R_std = dill.load(f)
    
#with open('/home/lucas/tesis2/test_outputs/times_sequence_tree_lightcurves.R.txt_5000samples_semistdTrue_stdFalse_20levels_50samples_per_class_20results_per_query.dill', 'rb') as f:
#    test_results_5000_R_semi = dill.load(f)
    

In [25]:
def aggregate_results(results):
    aggregated_results = []
    for  class_, class_results in results.items():
        aggregated_results += list(class_results.values())
    return aggregated_results

def aggregate_results_fixed_n(model_results, n):
    aggregated_results = {}
    for class_name, results in model_results.items():
        nscores = [scores[n] for scores in results.values()]
        aggregated_results[class_name] = nscores
    return aggregated_results

In [26]:
def plot_scores(names, results, title):
    data = []
    for name, result in zip(names, results):
        x = []
        y = []
        for query in result:
            x += range(1, len(query) + 1)
            y += query.tolist()
        x = ['.{0}.'.format(i) for i in x]
        trace = go.Box(
            y=y,
            x=x,
            name=name,
            boxmean='sd'
        )
        data.append(trace)
        
    layout = go.Layout(
        yaxis=dict(
            title=title,
            zeroline=True
        ),
        boxmode='group'
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
    
def plot_classes(results_dict, title):
    data = []
    for model_name, result in results_dict.items():
        x = []
        y = []
        for class_name, scores in sorted(result.items()):
            x += len(scores)*[class_name]
            y += scores
        trace = go.Box(
            y=y,
            x=x,
            name=model_name,
            boxmean=True
        )
        data.append(trace)
    layout = go.Layout(
        yaxis=dict(
            title=title,
            zeroline=True
        ),
        boxmode='group'
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [28]:
models = [test_results_100_R, test_results_500_R, test_results_1000_R, test_results_5000_R, fats_pca2, mac_feats,
          test_results_100_R_semi, test_results_500_R_semi, test_results_1000_R_semi, test_results_5000_R_semi,
           test_results_500_R_std]
models_results_by_class = [model['results'] for model in models]
models_names = ['100_R', '500_R', '1000_R', '5000_R', 'fats_pca2', 'mac_feats', 'test_results_100_R_semi',
               'test_results_500_R_semi', 'test_results_1000_R_semi', 'test_results_5000_R_semi',
                'test_results_500_R_std']
aggregated_results = [aggregate_results(result) for result in models_results_by_class]
plot_scores(models_names, aggregated_results, 'nDCG')

In [29]:
model = models_results_by_class[1]
class_names, results = zip(*((class_name,  list(results.values())) for class_name, results in model.items()))
plot_scores(class_names, results, 'nDCG')

In [13]:
model = models_results_by_class[0]
class_names, results = zip(*((class_name,  list(results.values())) for class_name, results in model.items()))
plot_scores(class_names, results, 'nDCG')

In [ ]:
n = 20
results_by_class = {name: aggregate_results_fixed_n(model, n - 1)
                    for model, name in zip(models_results_by_class, models_names)}
plot_classes(results_by_class, 'nDCG')

In [ ]:
results_by_class['1000']['BE']

In [ ]:
import dill
with open('/home/lucas/tesis2/models/sequence_tree_lightcurves.B.txt_500samples_20levels.dill', 'rb') as f:
    st = dill.load(f)

In [ ]:
with open('/home/lucas/tesis2/models/sequence_tree_lightcurves.B.txt_500samples_20levels.dill', 'rb') as f:
    st2 = dill.load(f)

In [ ]:
st.root.children